In [25]:
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image
import torch
import gradio
import numpy as np


In [2]:
pipeline = AutoPipelineForInpainting.from_pretrained("diffusers/stable-diffusion-xl-1.0-inpainting-0.1", torch_dtype=torch.float16).to("cuda")
pipeline.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name="ip-adapter_sdxl.bin")
pipeline.set_ip_adapter_scale(0.6)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

The config attributes {'decay': 0.9999, 'inv_gamma': 1.0, 'min_decay': 0.0, 'optimization_step': 37000, 'power': 0.6666666666666666, 'update_after_step': 0, 'use_ema_warmup': False} were passed to UNet2DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [20]:
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/ip_adapter_mask.png")
image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/ip_adapter_bear_1.png")
ip_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/ip_adapter_gummy.png")

generator = torch.Generator(device="cpu").manual_seed(4)
images = pipeline(
    prompt="a cute gummy bear waving",
    image=image,
    mask_image=mask_image,
    ip_adapter_image=ip_image,
    generator=generator,
    num_inference_steps=100,
).images


  0%|          | 0/99 [00:00<?, ?it/s]

In [ ]:
interface = gradio.ImageMask()

In [38]:
import gradio as gr
import time
from PIL import Image
from diffusers.image_processor import IPAdapterMaskProcessor


output_height = 1024
output_width = 1024

processor = IPAdapterMaskProcessor()

with gr.Blocks() as demo:
    im = gr.ImageMask(
        type="pil",
        crop_size="1:1",
    )

    with gr.Group():
        with gr.Row():
            im_out_1 = gr.Image(type="pil")
            im_out_2 = gr.Image(type="pil")
            im_out_3 = gr.Image(type="pil")

    btn = gr.Button("sleep")
    @btn.click(inputs=im, outputs=[im_out_1, im_out_2, im_out_3])
    def sleep(im):
        #time.sleep(5)
        
        mask = im["layers"][0]
        mask = processor.preprocess([mask], height=output_height, width=output_width)[0]
        print(np.asarray(mask).shape)
        image = im["background"]
        ip_image = load_image("/home/z004e29c/Pictures/Screenshots/logo.png")
        generator = torch.Generator(device="cpu").manual_seed(4)
        images = pipeline(
            prompt="a cute gummy bear waving",
            image=image,
            mask_image=mask_image,
            ip_adapter_image=ip_image,
            generator=generator,
            num_inference_steps=100,
        ).images
        

        return [im["background"], im["layers"][0], images[0]]

    
    #im.change(sleep, outputs=[im_out_1, im_out_2, im_out_3, im_out_4], inputs=im)

if __name__ == "__main__":
    demo.launch()



Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


(1, 1024, 1024)


Traceback (most recent call last):
  File "/home/z004e29c/envs/pytorch/lib/python3.10/site-packages/gradio/queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
  File "/home/z004e29c/envs/pytorch/lib/python3.10/site-packages/gradio/route_utils.py", line 233, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/z004e29c/envs/pytorch/lib/python3.10/site-packages/gradio/blocks.py", line 1608, in process_api
    result = await self.call_function(
  File "/home/z004e29c/envs/pytorch/lib/python3.10/site-packages/gradio/blocks.py", line 1176, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/z004e29c/envs/pytorch/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/home/z004e29c/envs/pytorch/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 2144, in run_sync_in_worker_thread
    return await 

In [36]:
np.asarray(mask).shape

NameError: name 'mask' is not defined